In [1]:
from model import G_NET
from dataset import MultimodalCUB
from trainer import condGANTrainer

from torchvision import transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import torch

import os
import numpy as np

In [2]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print(device)

cuda:0


In [3]:
torch.device(0)

device(type='cuda', index=0)

In [4]:
# CONFIGS
EMBEDDING_DIM = 128
Z_DIM = 100
GF_DIM = 64
TEXT_DIM = 1024
BRANCH_NUM = 3
BASE_SIZE = 32
BATCH_SIZE = 16
EPOCH = 1
TEST_B_EXAMPLE = True
ITERATION = 2

### Defining and loading model

In [5]:
model = G_NET(gf_dim=GF_DIM, branch_num=BRANCH_NUM, ef_dim=EMBEDDING_DIM, t_dim=TEXT_DIM, z_dim=Z_DIM)

In [6]:
model = nn.DataParallel(model)

In [7]:
model_path = os.path.join(os.getcwd(), 'pretrainedModels', 'netG_210000.pth')

In [8]:
model.load_state_dict(torch.load(model_path, map_location=device))

<All keys matched successfully>

In [9]:
model

DataParallel(
  (module): G_NET(
    (ca_net): CA_NET(
      (fc): Linear(in_features=1024, out_features=512, bias=True)
      (relu): GLU()
    )
    (h_net1): INIT_STAGE_G(
      (fc): Sequential(
        (0): Linear(in_features=228, out_features=32768, bias=False)
        (1): BatchNorm1d(32768, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): GLU()
      )
      (upsample1): Sequential(
        (0): Upsample(scale_factor=2.0, mode=nearest)
        (1): Conv2d(1024, 1024, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (2): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (3): GLU()
      )
      (upsample2): Sequential(
        (0): Upsample(scale_factor=2.0, mode=nearest)
        (1): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (2): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (3): GLU()
      )
      (u

### loading the dataset and preprocessing

In [10]:
imsize = BASE_SIZE * (2 ** (BRANCH_NUM-1))
image_transform = transforms.Compose([
        transforms.Resize(int(imsize * 76 / 64)),
        transforms.RandomCrop(imsize),
        transforms.RandomHorizontalFlip()])

In [11]:
test_dataset = MultimodalCUB(data_dir=os.getcwd(), branch_num=BRANCH_NUM,split='test', transform=image_transform)

Total filenames:  11788 001.Black_footed_Albatross/Black_Footed_Albatross_0046_18.jpg
Load filenames from: C:\Users\ani_1\PycharmProjects\Thesis\test\filenames.pickle (2933)
embeddings:  (2933, 10, 1024)


In [12]:
dataloader = torch.utils.data.DataLoader(
        test_dataset, batch_size=BATCH_SIZE, drop_last=True, shuffle=False)

In [13]:
test_dataset.prepair_test_pairs(0)

([tensor([[[-0.9843, -0.9843, -0.9843,  ..., -0.9608, -0.9765, -0.9686],
           [-0.9843, -0.9843, -0.9843,  ..., -0.8510, -0.9608, -0.9686],
           [-0.9765, -0.9686, -0.9843,  ..., -0.6235, -0.8431, -0.9686],
           ...,
           [-0.9765, -0.9765, -0.9843,  ...,  0.1922,  0.1451,  0.0588],
           [-0.9686, -0.9765, -0.9765,  ...,  0.1451,  0.1373,  0.1529],
           [-0.9765, -0.9765, -0.9843,  ...,  0.1137,  0.1373,  0.0588]],
  
          [[-0.3176, -0.3020, -0.2941,  ..., -0.1843, -0.2157, -0.2314],
           [-0.3176, -0.3098, -0.2941,  ..., -0.1294, -0.1843, -0.2157],
           [-0.3098, -0.2941, -0.2784,  ..., -0.0667, -0.1373, -0.1765],
           ...,
           [-0.3255, -0.3098, -0.3020,  ..., -0.0745, -0.1137, -0.2000],
           [-0.3098, -0.3020, -0.2941,  ..., -0.1216, -0.1137, -0.0980],
           [-0.3020, -0.2941, -0.2941,  ..., -0.1686, -0.1216, -0.1922]],
  
          [[ 0.2000,  0.2235,  0.2392,  ...,  0.3412,  0.3255,  0.3176],
           

In [14]:
trainer = condGANTrainer(output_dir=os.getcwd(), 
                         data_loader=dataloader, 
                         imsize=BASE_SIZE, 
                         is_train=False, 
                         batch_size=BATCH_SIZE,
                        epoch=1, model=model,
                        device=device)

In [15]:
trainer.evaluate(split_dir='test', z_dim=Z_DIM, test_b_example=TEST_B_EXAMPLE, it=ITERATION)

C:\ProgramData\Anaconda3\envs\pytorch\lib\site-packages\torch\nn\functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Make a new folder:  C:\Users\ani_1\PycharmProjects\Thesis\generatedImages/iteration2/super/valid/001.Black_footed_Albatross
Make a new folder:  C:\Users\ani_1\PycharmProjects\Thesis\generatedImages/iteration2/super/valid/004.Groove_billed_Ani
Make a new folder:  C:\Users\ani_1\PycharmProjects\Thesis\generatedImages/iteration2/super/valid/006.Least_Auklet
Make a new folder:  C:\Users\ani_1\PycharmProjects\Thesis\generatedImages/iteration2/super/valid/008.Rhinoceros_Auklet
Make a new folder:  C:\Users\ani_1\PycharmProjects\Thesis\generatedImages/iteration2/super/valid/009.Brewer_Blackbird
Make a new folder:  C:\Users\ani_1\PycharmProjects\Thesis\generatedImages/iteration2/super/valid/014.Indigo_Bunting
Make a new folder:  C:\Users\ani_1\PycharmProjects\Thesis\generatedImages/iteration2/super/valid/023.Brandt_Cormorant
Make a new folder:  C:\Users\ani_1\PycharmProjects\Thesis\generatedImages/iteration2/super/valid/029.American_Crow
Make a new folder:  C:\Users\ani_1\PycharmProjects\Thesis